алгоритм https://epubs.siam.org/doi/pdf/10.1137/1.9781611972771.42

In [1]:
import pandas as pd
import numpy as np
import json
import os
import math
import surprise
import dateutil.relativedelta
from datetime import timezone
from cornac.metrics import MAE, RMSE, Precision, Recall, NDCG, AUC, MAP
from sklearn.metrics import mean_squared_error, mean_absolute_error
from surprise import SVD, NMF
from surprise import Reader, Dataset
import datetime

FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.


In [2]:
#подготовка данных
bounds = {'before_covid':[datetime.datetime(2020,1,1), datetime.datetime(2020,3,20)],
          'during_covid':[datetime.datetime(2020,3,20), datetime.datetime(2020,8,1)], 
          'after_covid':[datetime.datetime(2020,8,1), datetime.datetime(2020,10,1)]}
def func(date):
    if date<=bounds['before_covid'][1]:
        return 0
    if bounds['during_covid'][0]<=date<=bounds['during_covid'][1]:
        return 1
    if bounds['after_covid'][0]<=date:
        return 2
def add_new_columns_with_context(df):
    df = df.sort_values('date', ascending=False)
    df.drop_duplicates(subset=['user_name', 'unique_rest'], inplace=True)
    df.fillna(0, inplace=True)
    for col in df.columns:
        if len(df[col].unique()) == 1:
            df.drop(col,inplace=True,axis=1)
    df.drop_duplicates(subset=['user_name', 'unique_rest'], inplace=True)
#оставляем пользователей с количеством отзывов >1
    users = df.groupby('user_name')['rating'].count()[df.groupby('user_name')['rating'].count()>1].index.tolist()
    df = df[df.user_name.isin(users)]
    q_dict = df.drop_duplicates('unique_rest').groupby('city')['dist'].describe()[['25%', '50%', '75%']]
    q_df = q_dict.to_dict('index')
    #название городов в числа
    city2cat_dict = {x:i for i,x in enumerate(df['city'].unique().tolist())}
    print(city2cat_dict)
    df['city'] = df['city'].apply(lambda x: city2cat_dict[x])
    # текущее количество отзывов у пользователя
    df['covid'] = df.date.apply(lambda x: func(x))
    return df
def for_city(df):
    #контекст отзыва
    #перцентиль по контексту
    df.drop(['text', 'rest_name', 'neg', 'neu', 'pos', 'review_language', 'average_rating',
         'num_all_rev',"country"], axis=1, inplace=True)
    norm = ['user_name','unique_rest','date']
    for col in df.columns:
        if col not in norm:
            df[col] = df[col].astype(int)
    return df

In [3]:
def create_train_test(df,date_start,date_end):
    df_train = df[(df.date.between(date_start-dateutil.relativedelta.relativedelta(months=6), 
                                   date_start-dateutil.relativedelta.relativedelta(days=1)))]
    df_test = df[(df.date.between(date_start, date_end))]
    #пользователи и рестораны только те которые в трейне
    df_test = df_test[(df_test.user_name.isin(df_train.user_name.unique().tolist()))
                       &(df_test.unique_rest.isin(df_train.unique_rest.unique().tolist()))]
    return df_train,df_test

In [4]:
def m_calculate(n1,n2): 
    return 1/(1/n1+1/n2) 
def delta_calculate(delta,n):
    return delta/n
def epsilon_calculate(m,delta): 
    return math.sqrt((1/(2*m))*math.log(4/delta))
def checking_stop_condition(train,date_group,border,delta):
    condition_value=1
    for i in range(1,int(border/5)-1):
        train1=train[train.date_new<date_group-i*5]
        train2=pd.concat([train,train1]).drop_duplicates(keep=False)
        n1=len(train1)
        n2=len(train2)
        mu1=train1.rate.mean()
        mu2=train2.rate.mean()
        try:
            eps=epsilon_calculate(m_calculate(n1,n2),delta_calculate(delta,n1+n2))
            if(abs(mu1-mu2)>=eps):
                condition_value=condition_value*0
        except:
            eps=0
    return condition_value
def find_train_set(train_all,date_group,border,delta):
    train=train_all[train_all.date<date_group]
    train_new=pd.DataFrame()
    window_size=0
    for i in range(1,int((31*border/3))):
        train_new=train[train.date>date_group-np.timedelta64(31*border, 'D')+np.timedelta64(3*i, 'D')]
        window_size=len(train_new.date.unique())
        if(checking_stop_condition(train_new,date_group,border,delta)):
            break
    return train_new,window_size   


In [5]:
def addaptive_window_alg(df,bounds,city,border,delta):
    results_surprise = pd.DataFrame()
    ndcg = NDCG(10)
    all_pred = pd.DataFrame()
    for i,k in enumerate(bounds.keys()):
        print(bounds[k][0], bounds[k][1])
        df_train,df_test=create_train_test(df,bounds[k][0], bounds[k][1])
        df_train=df_train[['user_name', 'unique_rest', 'rating',"date"]]
        df_test=df_test[['user_name', 'unique_rest', 'rating','date']]
        test_covid=df_test
        train_all=df
        groups=test_covid.date.unique()
        ndcg_metric = {}
        met = pd.DataFrame()
        for date_group in groups:
            test=test_covid[test_covid.date==date_group]
            train,window_size=find_train_set(train_all,date_group,border,delta)
            #print(date_group,window_size)
    #print(len(train))
            df_train=train[['user_name', 'unique_rest', 'rating']]
            df_test=test[['user_name', 'unique_rest', 'rating']]
            reader = Reader(rating_scale=(1, 5))
            data_do_train = Dataset.load_from_df(df_train, reader)
            trainset = data_do_train.build_full_trainset()
            algo = SVD()
            algo.fit(trainset)
            df_test_ = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
            df_test_ = df_test_[df_test_.index.isin(df_test.user_name.unique().tolist())]
            df_test_.update(df_test.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0))
            df_train_ = df_train.pivot(index='user_name', columns='unique_rest', values='rating').fillna(0)
            df_train_ = df_train_[df_train_.index.isin(df_train.user_name.unique().tolist())]
            df_test_dict = df_test_.T.to_dict('list')
    #df_test_dict_all
            us = {}
    #print(df_test_.index.tolist())
            for u in df_test_.index.tolist():
                values = [algo.predict(u, i).est if df_train_.loc[u, i]==0 else 0 for i in df_test_.columns.tolist()]
                us[u] = values
            for kk in us.keys():
                val=ndcg.compute(df_test_dict[kk], np.array(us[kk]).argsort()[::-1])
                if (not math.isnan(val)):
                    ndcg_metric[kk] = val
        ndcg_ = np.mean(list(ndcg_metric.values()))
        met = met.append({
            'alg':"SVD c аддаптивным окном",
            'period': (str(bounds[k][0])[:7]+' - '+str(bounds[k][1])[:7]),
            "city":city,
        #'rmse': rmse,
        #'mae': mae,
            'ndcg10': ndcg_,
            }, ignore_index=True)
        all_pred = all_pred.append(met, ignore_index=True)
    return all_pred

In [6]:
df = pd.read_csv('C:\Anaconda3\Scripts\Диплом\ONLINE EXCPERIMENT\GIT/FINAL_REVWS_ALL.csv', index_col=0)
df['date'] = pd.to_datetime(df['date'])

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df_all=df.copy()
cities=['Los', 'NY','Rome','Stockholm','saint']
df_all=df_all[df_all.city.isin(cities)]
df_all=add_new_columns_with_context(df_all)

{'Rome': 0, 'NY': 1, 'Stockholm': 2, 'Los': 3, 'saint': 4}


In [44]:
all_city_pred=pd.DataFrame()
border=2
delta=0.5
for city in range(0,len(cities)):
    print(city)
    df=df_all[df_all.city==city]
    df=df.reset_index(drop=True)
    df=for_city(df)
    window_method=addaptive_window_alg(df,bounds,city,border,delta)
    all_city_pred=pd.concat([all_city_pred,window_method], ignore_index=True)
    all_city_pred.to_csv("C:\Anaconda3\Scripts\Диплом\ONLINE EXCPERIMENT\GIT\Адаптивное окно/"
                         +"all_city_adaptive"+str(border)+".csv",sep=";")

0
2020-01-01 00:00:00 2020-03-20 00:00:00
2020-03-20 00:00:00 2020-08-01 00:00:00
2020-08-01 00:00:00 2020-10-01 00:00:00
1
2020-01-01 00:00:00 2020-03-20 00:00:00
2020-03-20 00:00:00 2020-08-01 00:00:00
2020-08-01 00:00:00 2020-10-01 00:00:00
2
2020-01-01 00:00:00 2020-03-20 00:00:00
2020-03-20 00:00:00 2020-08-01 00:00:00
2020-08-01 00:00:00 2020-10-01 00:00:00
3
2020-01-01 00:00:00 2020-03-20 00:00:00
2020-03-20 00:00:00 2020-08-01 00:00:00
2020-08-01 00:00:00 2020-10-01 00:00:00
4
2020-01-01 00:00:00 2020-03-20 00:00:00
2020-03-20 00:00:00 2020-08-01 00:00:00
2020-08-01 00:00:00 2020-10-01 00:00:00


# Стандартные рекомендательгные алгоритмы

In [2]:
def svd_learning(train_set):
    reader=surprise.Reader(rating_scale=(train_set.rate.min(),train_set.rate.max()))
    data=surprise.Dataset.load_from_df(train_set,reader)
    alg=surprise.SVD()
    output=alg.fit(data.build_full_trainset())
    return alg
def svd_model(user_code,alg,upper_border,all_data,crd):
    pred=[]
    for place in all_data.place_code.unique():
        pred.append(alg.predict(uid=user_code,iid=place).est)   
    svd_rating=dict(zip(all_data.place_code.unique(),pred)) 
    svd_rating={k: v for k, v in sorted(svd_rating.items(), key=lambda item: item[1],reverse=True)}
    return svd_rating
def svd_crd_model(user_code,alg,upper_border,all_data,crd):
    pred=[]
    for place in all_data.place_code.unique():
        pred.append(alg.predict(uid=user_code,iid=place).est+crd)   
    svd_rating=dict(zip(all_data.place_code.unique(),pred)) 
    svd_rating={k: v for k, v in sorted(svd_rating.items(), key=lambda item: item[1],reverse=True)}
    return svd_rating
def coll_filt(train_set):
    reader=surprise.Reader(rating_scale=(train_set.rate.min(),train_set.rate.max()))
    data=surprise.Dataset.load_from_df(train_set,reader)
    sim_options = {'name': 'cosine',
               'user_based':True  # compute  similarities between items
               }
    alg=surprise.KNNWithMeans(sim_options=sim_options)
    output=alg.fit(data.build_full_trainset())
    return alg
def baseline_learning(train_set):
    reader=surprise.Reader(rating_scale=(train_set.rate.min(),train_set.rate.max()))
    data=surprise.Dataset.load_from_df(train_set,reader)
    alg=surprise.prediction_algorithms.baseline_only.BaselineOnly()
    output=alg.fit(data.build_full_trainset())
    return alg
def normpred_learning(train_set):
    reader=surprise.Reader(rating_scale=(train_set.rate.min(),train_set.rate.max()))
    data=surprise.Dataset.load_from_df(train_set,reader)
    alg=surprise.prediction_algorithms.random_pred.NormalPredictor()
    output=alg.fit(data.build_full_trainset())
    return alg    
def nmf_learning(train_set):
    reader=surprise.Reader(rating_scale=(train_set.rate.min(),train_set.rate.max()))
    data=surprise.Dataset.load_from_df(train_set,reader)
    alg=surprise.NMF()
    output=alg.fit(data.build_full_trainset())
    return alg    
def slope_learning(train_set):
    reader=surprise.Reader(rating_scale=(train_set.rate.min(),train_set.rate.max()))
    data=surprise.Dataset.load_from_df(train_set,reader)
    alg=surprise.prediction_algorithms.slope_one.SlopeOne()
    output=alg.fit(data.build_full_trainset())
    return alg
def coclust_learning(train_set):
    reader=surprise.Reader(rating_scale=(train_set.rate.min(),train_set.rate.max()))
    data=surprise.Dataset.load_from_df(train_set,reader)
    alg=surprise.prediction_algorithms.co_clustering.CoClustering()
    output=alg.fit(data.build_full_trainset())
    return alg

In [3]:
def create_global_real_pred(real_glob,pred_glob,crd,test,alg_svd,coll,bl_alg,np_alg,slope_alg,coclust_alg,nmf_alg):
    real_glob[0],pred_glob[0]=rmse_for_alg(test,alg_svd,svd_model,crd,real_glob[0],pred_glob[0])
    real_glob[1],pred_glob[1]=rmse_for_alg(test,coll,svd_model,crd,real_glob[1],pred_glob[1])
    real_glob[2],pred_glob[2]=rmse_for_alg(test,bl_alg,svd_model,crd,real_glob[2],pred_glob[2])
    real_glob[3],pred_glob[3]=rmse_for_alg(test,np_alg,svd_model,crd,real_glob[3],pred_glob[3])
    real_glob[4],pred_glob[4]=rmse_for_alg(test,nmf_alg,svd_model,crd,real_glob[4],pred_glob[4])
    real_glob[5],pred_glob[5]=rmse_for_alg(test,slope_alg,svd_model,crd,real_glob[5],pred_glob[5])
    real_glob[6],pred_glob[6]=rmse_for_alg(test,coclust_alg,svd_model,crd,real_glob[6],pred_glob[6])
    real_glob[7],pred_glob[7]=rmse_for_alg(test,alg_svd,svd_crd_model,crd,real_glob[7],pred_glob[7])
    return real_glob,pred_glob
def rmse_for_alg(test,alg,svd_model,crd,real,pred):
    r_p_one=rmse_for(test,alg,svd_model,crd)
    real.extend(r_p_one[0])
    pred.extend(r_p_one[1])
    return real,pred
def rmse_for(test_set,alg,svd_model,crd):
    pred,real=[],[]
    for user in test_set.user_code.unique():
        rec_for_user=svd_model(user,alg,len(data.place_code.unique()),data,crd)
        real,pred=real_pred(test_set,user,rec_for_user,pred,real)
    return real,pred
def real_pred(test_set,user,rec_for_user,pred,real):
    test_per_user=test_set[test_set.user_code==user]
    places=test_per_user.place_code
    rates=test_per_user.rate
    d=dict(zip(places,rates))
    for i in d:
        pred.append(rec_for_user[i])
        real.append(d[i]) 
    return real,pred
def rmse(real,pred):
    for_i=0
    for i in range(len(real)):
        for_i+=(real[i]-pred[i])**2
    return math.sqrt(for_i/len(real))

# Адаптивное скользящее окно

In [12]:
def m_calculate(n1,n2): 
    return 1/(1/n1+1/n2) 
def delta_calculate(delta,n):
    return delta/n
def epsilon_calculate(m,delta): 
    return math.sqrt((1/(2*m))*math.log(4/delta))

In [13]:
def checking_stop_condition(train,date_group,border,delta):
    condition_value=1
    for i in range(1,int(border/5)-1):
        train1=train[train.date_new<date_group-i*5]
        train2=pd.concat([train,train1]).drop_duplicates(keep=False)
        n1=len(train1)
        n2=len(train2)
        mu1=train1.rate.mean()
        mu2=train2.rate.mean()
        try:
            eps=epsilon_calculate(m_calculate(n1,n2),delta_calculate(delta,n1+n2))
            if(abs(mu1-mu2)>=eps):
                condition_value=condition_value*0
        except:
            eps=0
    return condition_value

In [14]:
def find_train_set(train_all,date_group,border,delta):
    train=train_all[train_all.date_new<date_group]
    for i in range(1,int((border/3))):
        train_new=train[train.date_new>date_group-border+3*i]
        window_size=len(train_new.date_new.unique())
        if(checking_stop_condition(train_new,date_group,border,delta)):
            break
    return train_new,window_size  

In [4]:
def addaptive_window(train_all,test_covid,border,delta):
    rmse_global=[]
    real_glob=[[],[],[],[],[],[],[],[]]
    pred_glob=[[],[],[],[],[],[],[],[]]
    groups=test_covid.date_new.unique()
    for date_group in groups:
        test=test_covid[test_covid.date_new==date_group]
        train,window_size=find_train_set(train_all,date_group,border,delta)
        print(date_group,window_size)
        covid_mean=test.rate.mean()
        without_covid=train[train.date_new<(1585318575-min_date)/(60*60*24)]
        usual_mean=without_covid.rate.mean()
        crd=covid_mean-usual_mean
        if(np.isnan(crd)):
            crd=0
        alg_svd=svd_learning(train[["user_code","place_code","rate"]])
        coll=coll_filt(train[["user_code","place_code","rate"]])
        bl_alg=baseline_learning(train[["user_code","place_code","rate"]])
        np_alg=normpred_learning(train[["user_code","place_code","rate"]])
        nmf_alg=nmf_learning(train[["user_code","place_code","rate"]])
        slope_alg=slope_learning(train[["user_code","place_code","rate"]])
        coclust_alg=coclust_learning(train[["user_code","place_code","rate"]])
        real_glob,pred_glob=create_global_real_pred(real_glob,pred_glob,crd,test,alg_svd,
                                                    coll,bl_alg,np_alg,slope_alg,coclust_alg,nmf_alg)
    for i in range(len(real_glob)):
        rmse_global.append(rmse(real_glob[i],pred_glob[i]))
    df=pd.DataFrame({
        "method":["svd","coll","bl_alg","np_alg","nmf_alg","slope","coclust","scd+crd"],
        "rmse":rmse_global 
    })
    return df

In [5]:
def train_test(data,period):
    min_date=data.date.min()
    data["date_new"]=[(date-min_date)/(60*60*24) for date in data.date]
    train_all=data
    #во время
    if (period=="in_time"):
        test_covid=data[data["date"]>=1585318575]
        test_covid=test_covid[test_covid["date"]<=1598537775]
    #после
    if(period=="after"):
        test_covid=data[data["date"]<=1601501263]#1 октября
        test_covid=test_covid[test_covid["date"]>=1596230863]#1 августа
    #до
    if(period=="before"):
        test_covid=data[data["date"]>=1577827663]
        test_covid=test_covid[test_covid["date"]<=1584653263]
    return train_all,test_covid

In [8]:
def to_df(path,data,period,border,delta):
    train_all,test_covid=train_test(data,period)
    df=addaptive_window(train_all,test_covid,border,delta)
    df.to_csv(path+"/"+path+"_"+period+"_адаптивный оконный метод.csv",sep=";")
    return df   

# Для данных по разным странам

In [ ]:
#для данных по странам
pathes=["paris","istanbul","berlin","sankt-petersburg","london"]
filenames=[]
for i in pathes:
    filenames.append(os.listdir(i))
name="_data"
name_in_city=[]
for i in filenames:
    for j in i:
        if (name in j):
            name_in_city.append(j)

In [ ]:
border=31*2
delta=0.5
#во время ковид
for i in range(len(name_in_city)):
    path=pathes[i]
    data=pd.read_csv(path+"/"+path+"_data.csv",sep=";",index_col="Unnamed: 0")
    to_df(path,data,"in_time",border,delta)